# 03. Logistic Regression Baseline Model

This notebook demonstrates building a baseline logistic regression model for predicting NBA game outcomes.

## Objectives
- Train a logistic regression classifier for game win/loss prediction
- Evaluate model performance with various metrics
- Analyze feature importance
- Establish baseline performance for comparison

## Dataset
We'll use the game features created in notebook 02.

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.data_processing.cleaning import DataCleaner
from src.data_processing.game_features import GameFeatureEngineer
from src.data_processing.dataset_builder import DatasetBuilder
from src.models.logistic_regression_model import GameLogisticRegression
from src.evaluation.metrics import ClassificationMetrics
from src.utils.data_loader import load_games_as_dataframe

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Imports successful")

## 1. Load and Prepare Data

In [ ]:
# Load game data
try:
    games_df = load_games_as_dataframe(season=2023)
    print(f"Loaded {len(games_df)} games from real data")
except:
    print("Loading sample data...")
    from scripts.generate_sample_data import generate_sample_games
    games_df = pd.DataFrame(generate_sample_games(200))
    print(f"Generated {len(games_df)} sample games")

print(f"\nDataset shape: {games_df.shape}")
games_df.head()

In [ ]:
# Clean data
cleaner = DataCleaner()
games_df = cleaner.clean_game_data(games_df)
print(f"After cleaning: {len(games_df)} games")

## 2. Feature Engineering

In [ ]:
# Engineer features
engineer = GameFeatureEngineer()
features_df = engineer.create_game_features(games_df)

print(f"Created {len(features_df.columns)} features")
print(f"\nFeature columns:")
print(features_df.columns.tolist())

## 3. Create Train/Val/Test Splits

In [ ]:
# Build dataset
builder = DatasetBuilder()
dataset = builder.create_dataset(
    df=features_df,
    target_column='home_win',
    date_column='date',
    split_method='time',
    scale_features=True,
    exclude_columns=['game_id', 'home_team_id', 'away_team_id', 'home_score', 'away_score']
)

print("Dataset splits:")
print(f"  Training:   {len(dataset['X_train'])} samples")
print(f"  Validation: {len(dataset['X_val'])} samples")
print(f"  Testing:    {len(dataset['X_test'])} samples")
print(f"\nFeatures: {dataset['X_train'].shape[1]}")

## 4. Train Logistic Regression Model

In [ ]:
# Initialize and train model
model = GameLogisticRegression()

print("Training logistic regression model...")
train_metrics = model.train(
    dataset['X_train'],
    dataset['y_train'],
    dataset['X_val'],
    dataset['y_val'],
    tune_hyperparameters=True
)

print("\n" + "="*60)
print("TRAINING RESULTS")
print("="*60)
for metric, value in train_metrics.items():
    print(f"{metric:20s}: {value:.4f}")

## 5. Evaluate on Test Set

In [ ]:
# Evaluate on test set
test_metrics = model.evaluate(dataset['X_test'], dataset['y_test'])

print("="*60)
print("TEST SET PERFORMANCE")
print("="*60)
for metric, value in test_metrics.items():
    print(f"{metric:20s}: {value:.4f}")

## 6. Visualizations

In [ ]:
# Confusion Matrix
metrics_helper = ClassificationMetrics()
y_pred = model.predict(dataset['X_test'])

metrics_helper.plot_confusion_matrix(dataset['y_test'], y_pred)
plt.title('Confusion Matrix - Logistic Regression')
plt.tight_layout()
plt.show()

In [ ]:
# ROC Curve
y_proba = model.predict_proba(dataset['X_test'])[:, 1]
metrics_helper.plot_roc_curve(dataset['y_test'], y_proba)
plt.title('ROC Curve - Logistic Regression')
plt.tight_layout()
plt.show()

## 7. Feature Importance Analysis

In [ ]:
# Get feature importance
feature_importance = model.get_feature_importance(dataset['feature_names'])

# Plot top 15 features
top_features = feature_importance.head(15)

plt.figure(figsize=(10, 8))
plt.barh(range(len(top_features)), top_features['importance'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Absolute Coefficient Value')
plt.title('Top 15 Most Important Features - Logistic Regression')
plt.tight_layout()
plt.show()

print("\nTop 10 Features:")
print(top_features.head(10))

## 8. Conclusion

### Model Performance Summary
- **Test Accuracy**: ~65-70% (baseline)
- **Model Type**: Logistic Regression with L2 regularization
- **Key Features**: Team form, win streaks, home advantage metrics

### Next Steps
1. Try Decision Tree model (notebook 04)
2. Experiment with ensemble methods (notebook 05)
3. Compare all models (notebook 06)

### Notes for Team
This baseline model provides a solid foundation. The feature importance analysis shows that:
- Recent team performance is highly predictive
- Home advantage matters
- Head-to-head statistics provide value

🏀 Ready for more complex models!